# Imports

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist, fashion_mnist
import keras as kb

# Data loading

In [6]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d kylegraupe/skin-cancer-binary-classification-dataset

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kylegraupe/skin-cancer-binary-classification-dataset
License(s): CC0-1.0
 22% 5.00M/22.5M [00:00<00:00, 39.6MB/s]
100% 22.5M/22.5M [00:00<00:00, 110MB/s] 


In [7]:
!unzip -qq /content/skin-cancer-binary-classification-dataset.zip

### Directory formatting


In [8]:
import os
import shutil
cancer_train_dir = "/content/Skin_Data/Cancer/Training"
cancer_test_dir = "/content/Skin_Data/Cancer/Testing"
noncancer_train_dir = "/content/Skin_Data/Non_Cancer/Training"
noncancer_test_dir = "/content/Skin_Data/Non_Cancer/Testing"

new_cancer_train_dir = "/content/Skin_Data/Training/Cancerous"
new_cancer_test_dir = "/content/Skin_Data/Testing/Cancerous"
new_noncancer_train_dir = "/content/Skin_Data/Training/Non-Cancerous"
new_noncancer_test_dir = "/content/Skin_Data/Testing/Non-Cancerous"

os.makedirs(new_cancer_train_dir, exist_ok=True)
os.makedirs(new_cancer_test_dir, exist_ok=True)
os.makedirs(new_noncancer_train_dir, exist_ok=True)
os.makedirs(new_noncancer_test_dir, exist_ok=True)

for file_name in os.listdir(cancer_train_dir):
  shutil.move(os.path.join(cancer_train_dir, file_name), new_cancer_train_dir)

for file_name in os.listdir(cancer_test_dir):
  shutil.move(os.path.join(cancer_test_dir, file_name), new_cancer_test_dir)

for file_name in os.listdir(noncancer_train_dir):
  shutil.move(os.path.join(noncancer_train_dir, file_name), new_noncancer_train_dir)

for file_name in os.listdir(noncancer_test_dir):
  shutil.move(os.path.join(noncancer_test_dir, file_name), new_noncancer_test_dir)


In [9]:
train_dir = "/content/Skin_Data/Training"
test_dir = "/content/Skin_Data/Testing"

### Loading into Keras

In [22]:
batch_size = 16
img_height = 170
img_width = 170
train_ds = kb.utils.image_dataset_from_directory(
    train_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = kb.utils.image_dataset_from_directory(
    test_dir,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 84 files belonging to 2 classes.
Found 204 files belonging to 2 classes.


# CNN Model


In [27]:
model = kb.Sequential()

model.add(kb.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(kb.layers.BatchNormalization())
model.add(kb.layers.MaxPooling2D((2, 2)))

model.add(kb.layers.Conv2D(16, (3, 3), activation='relu', kernel_regularizer=kb.regularizers.l2(0.001)))
model.add(kb.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=kb.regularizers.l2(0.001)))
model.add(kb.layers.MaxPooling2D((2, 2)))

model.add(kb.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=kb.regularizers.l2(0.001)))
model.add(kb.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=kb.regularizers.l2(0.001)))
model.add(kb.layers.MaxPooling2D((2, 2)))

model.add(kb.layers.Flatten())
model.add(kb.layers.Dense(64, activation='relu'))
model.add(kb.layers.Dropout(0.5))
model.add(kb.layers.Dense(1, activation='sigmoid'))

lr_schedule = kb.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_steps=35, decay_rate=0.8)
model.compile(optimizer=kb.optimizers.Adam(learning_rate=lr_schedule), loss='binary_crossentropy', metrics=['accuracy', kb.metrics.Precision(), kb.metrics.Recall()])

early_stopping = kb.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#lr_scheduler = kb.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history = model.fit(train_ds, epochs=12, validation_data=test_ds, callbacks=[early_stopping])

Epoch 1/12
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5197 - loss: 1.3204 - precision_2: 0.5443 - recall_2: 0.4715 - val_accuracy: 0.7941 - val_loss: 0.7042 - val_precision_2: 0.7941 - val_recall_2: 1.0000
Epoch 2/12
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6077 - loss: 0.8706 - precision_2: 0.5738 - recall_2: 0.7543 - val_accuracy: 0.2549 - val_loss: 0.9488 - val_precision_2: 1.0000 - val_recall_2: 0.0617
Epoch 3/12
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.7077 - loss: 0.6917 - precision_2: 0.7679 - recall_2: 0.5381 - val_accuracy: 0.7892 - val_loss: 0.6468 - val_precision_2: 0.8148 - val_recall_2: 0.9506
Epoch 4/12
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7812 - loss: 0.6718 - precision_2: 0.7270 - recall_2: 0.9315 - val_accuracy: 0.2549 - val_loss: 1.2503 - val_precision_2: 1.0000 - val_recall_2: 0.0617
Epoch 5/12
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7346 - loss: 0.6258 - precision_2: 0.7204 - recall_2: 0.7031 - val_accuracy: 0.789

# Transfer Learning Model

In [29]:
from tensorflow.keras.applications import EfficientNetB0

In [30]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
model = kb.Sequential()
model.add(base_model)
model.add(kb.layers.Flatten())
model.add(kb.layers.Dense(128, activation='relu'))
model.add(kb.layers.Dropout(0.3))
model.add(kb.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=kb.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_ds, epochs=5, validation_data=test_ds)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 80s 5s/step - accuracy: 0.6834 - loss: 1.1269 - val_accuracy: 0.7941 - val_loss: 2.0638
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.7847 - loss: 1.3909 - val_accuracy: 0.3971 - val_loss: 4.4221
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.9044 - loss: 0.4297 - val_accuracy: 0.8529 - val_loss: 0.7451
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.9874 - loss: 0.0306 - val_accuracy: 0.7941 - val_loss: 4.4006
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 40s 4s/step - accuracy: 0.9082 - loss: 0.3762 - val_accuracy: 0.8039 - val_loss: 2.8992
